This notebook uses the network from the iNNvestigate Mnist example and compares the statistics, patterns and signals computed by the iNNvestigate and pytorch patternnet implementations. For the comparison to be more accurate the innvestigate layer inputs and outputs are used for the statistics computation of both frameworks.

In [1]:
import numpy as np
import innvestigate
import keras
import torch
import torch.nn as nn
import pandas as pd
import itertools

import sys
sys.path.append('../')
import networks
import patterns

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

Using TensorFlow backend.


# Get data

In [2]:
data_all = np.load(
    "/home/rabea/Documents/AgBall/patternnet/patterns/testing/mnist_images_for_patterns.npy")
data = data_all[:10]
print(data.shape)

# reshape for pytorch usage
data_py = torch.FloatTensor(data.transpose(0,3,1,2))

(10, 28, 28, 1)


# Create iNNvestigate network

In [3]:
PATTERN_TYPE = 'relu'

In [4]:
input_shape = (28, 28, 1)

model = keras.models.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation="relu", input_shape=input_shape),
    keras.layers.Conv2D(64, (3, 3), activation="relu"),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation="relu"),
    keras.layers.Dense(10, activation="softmax"),
])

model.compile(loss="categorical_crossentropy",
                  optimizer='sgd',
                  metrics=["accuracy"])

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               4719104   
_________________________________________________________________
dense_2 (Dense)              (None, 10)                5130      
Total params: 4,743,050
Trainable params: 4,743,050
Non-trainable params: 0
_________________________________________________________________


In [6]:
# set weights to pretrained weights
weights_loaded = np.load(
    "/home/rabea/Documents/AgBall/patternnet/patterns/testing/innvestigate_mnistnet_weights.npy")
model.set_weights(weights_loaded)

In [7]:
model_wo_softmax = innvestigate.utils.keras.graph.model_wo_softmax(model)

In [8]:
analyzer = innvestigate.create_analyzer("pattern.net", 
                                        model_wo_softmax,
                                        pattern_type=PATTERN_TYPE)

analyzer.fit(data, batch_size=10, verbose=1)

Epoch 1/1
1/1 [==============================] - 0s 365ms/step - loss: 4.0000 - broadcast_1_loss: 1.0000 - broadcast_2_loss: 1.0000 - broadcast_3_loss: 1.0000 - broadcast_4_loss: 1.0000


## Get innvestigate patterns

In [9]:
# get patterns
patterns_innvestigate = analyzer._patterns
# for pattern in patterns_innvestigate:
#     print(pattern.shape)

## Get innvestigate statistics

In [10]:
# get the list of ReluPattern Instances, 4 instances for the 4 layers
pattern_instances = analyzer.computer._pattern_instances[PATTERN_TYPE]

# get the statistics per layer
stats_innvestigate = []
for instance in pattern_instances:
    stats = instance.stats_dict
#     for key in stats:
#         print(stats[key].shape)
#     print()
    stats_innvestigate.append(stats)

## Get statistics layer

In [11]:
# get the statistics network to be able to access the input and output
# to the statistics computations
statistics_network = analyzer.computer._computers[0]
# statistics_network.summary()

## Get innvestigate layer inputs and outputs

In [12]:
# get innvestigate outputs
model_outputs = [layer.output for layer in model_wo_softmax.layers]
inp = model_wo_softmax.input
layer_models = [keras.models.Model(inp,out) for out in model_outputs]
# first model just input model, can be deleted from list
layer_models.pop(0)
# third model model that ends after maxpooling, can be deleted from list
layer_models.pop(2)
# now third model ends with flatten layer
layer_models.pop(2)

# now change activation of final layer to get outputs without relu
# after having changed activations need to save and load model to apply changes to the graph
mod_count = 1
linear_models = []
for mod in layer_models:
    layers_in_model = mod.layers
    # set all layers activations to relu and then change the last layers activation to linear
    # if only changing last layer activation to linear, then this layer will also have a linear
    # activation in the next bigger model
    for layer in mod.layers:
        if layer.__class__ == keras.layers.convolutional.Conv2D or \
                    layer.__class__ == keras.layers.core.Dense:
            layer.activation = keras.activations.relu
    mod.layers[-1].activation = keras.activations.linear
    
    mod.compile(optimizer="sgd", loss="categorical_crossentropy")
    mod.save("./_model_%i" %mod_count)
    lin_mod = keras.models.load_model("./_model_%i" %mod_count)
    mod_count += 1
    linear_models.append(lin_mod)

In [13]:
# get the actual output predictions for the created models
layer_outputs_innvestigate_wo_relu = []
for ind, mod in enumerate(layer_models):
    out_wo_relu = linear_models[ind].predict_on_batch(data)
    layer_outputs_innvestigate_wo_relu.append(torch.FloatTensor(out_wo_relu))

In [14]:
# get output and input of layers used for statistics computation
# for outputs without bias use Ys, for inputs use Xs, output with bias is saved in 
# layer_outputs_innvestigate_wo_relu

statistics_layers_all = statistics_network.layers
# list with the indices for the Xs and Ys layers
input_model_inds = [17, 20, 4, 5]
output_model_inds = [19, 22, 24, 26]

input_model_outputs = [statistics_layers_all[ind].output for ind in input_model_inds]
output_model_outputs = [statistics_layers_all[ind].output for ind in output_model_inds]

inp = statistics_network.input
input_models = []
output_models = []
for i in range(4):
    input_models.append(keras.models.Model(inp, input_model_outputs[i]))
    output_models.append(keras.models.Model(inp, output_model_outputs[i]))

In [15]:
inputs_innvestigate = []
outputs_innvestigate_wo_bias = []

for i in range(4):
    layer_inp = input_models[i].predict_on_batch(data)
    layer_out_wo_bias = output_models[i].predict_on_batch(data)
    
    inputs_innvestigate.append(torch.FloatTensor(layer_inp))
    outputs_innvestigate_wo_bias.append(torch.FloatTensor(layer_out_wo_bias))

In [16]:
# for i in range(4):
#     print("Input shape:", inputs_innvestigate[i].shape)
#     print("Output shapes:", outputs_innvestigate_wo_bias[i].shape, 
#           layer_outputs_innvestigate_wo_relu[i].shape)
#     print()

In [17]:
# since the input and output for the conv layers in inputs_innvestigate and 
# outputs_innvestigate_wo_bias are already reshaped to the dense maps
# the outputs with bias also have to be reshaped to the dense maps
# for this first construct dummy inputs
dummy_conv_inputs = [torch.FloatTensor(np.zeros((10,1,28,28))), 
                     torch.FloatTensor(np.zeros((10,32,26,26)))]
# kernel size the same for both conv layers
ks = [3,3]
# now get outputs as reshaped pytorch tensors
conv_outputs_wo_relu = [layer_outputs_innvestigate_wo_relu[0].permute(0,3,1,2),
                        layer_outputs_innvestigate_wo_relu[1].permute(0,3,1,2)]
# dense map for first conv layer
_, out_dense1 = patterns._conv_maps_to_dense(dummy_conv_inputs[0], 
                                                       conv_outputs_wo_relu[0], 
                                                       ks)
# dense map for second conv layer
_, out_dense2 = patterns._conv_maps_to_dense(dummy_conv_inputs[1], 
                                                       conv_outputs_wo_relu[1],
                                                       ks)

# now substitute the conv outputs with its dense maps
layer_outputs_innvestigate_wo_relu[0] = out_dense1
layer_outputs_innvestigate_wo_relu[1] = out_dense2

# Create pytorch Patternnet

In [18]:
# reshape weights_loaded
weights_py = []
for i in range(8):
    weight = weights_loaded[i]
    if len(weight.shape) == 2:
        weight = weights_loaded[i].transpose(1,0)
    elif len(weight.shape) == 4:
        weight = weights_loaded[i].transpose(3,2,0,1)
    weight = torch.FloatTensor(weight)
    weights_py.append(weight)
#     print(weight.shape)

In [19]:
# pytorch Mnist network class
class MnistNet(nn.Module):
    
    def __init__(self):
        super(MnistNet, self).__init__()
        
        input_height = 28
        input_width = 28
        num_channels = 1
        self.map_height = int(((input_height - 2) - 2) / 2)
        self.map_width = int(((input_width - 2) - 2) / 2)
        
        self.conv1 = nn.Conv2d(num_channels, 32, 3)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.pool = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(64 * self.map_width * self.map_height, 512)
        self.fc2 = nn.Linear(512, 10)
        self.relu = nn.ReLU()
        
        self.layers = [self.conv1, self.relu, self.conv2, self.relu, self.pool,
                       self.fc1, self.relu, self.fc2]
        
    def forward(self, x):
        
        # first conv layer
        x = self.relu(self.conv1(x))
        # second conv layer
        x = self.relu(self.conv2(x))
        # pooling
        x = self.pool(x)
        if self.pool.return_indices:
            x = x[0]
        x = x.view(-1, 64 * self.map_width * self.map_height)
        # first dense layer
        x = self.relu(self.fc1(x))
        # second dense layer
        x = self.fc2(x)
        
        return x

In [20]:

# initialize class
model_pytorch = MnistNet()

# set weights
weight_ind = 0
for layer in model_pytorch.layers:
    layerclass = layer.__class__.__name__
    if layerclass in ['Conv2d', 'Linear']:
        layer.weight.data = weights_py[weight_ind]
        layer.bias.data = weights_py[weight_ind+1]
        weight_ind += 2

patternnet_pytorch = networks.PatternNet(model_pytorch.layers)

In [21]:
# perform statistics computations in here with innvestigates layer outputs
# compute statistics based on innvestigate output 
ind = 0
for layer in patternnet_pytorch.backward_layers[::-1]:
    layerclass = layer.__class__.__name__
    if layerclass in ['PatternConv2d', 'PatternLinear']:
        stat = patterns.compute_statistics(inputs_innvestigate[ind],
                                       outputs_innvestigate_wo_bias[ind],
                                       layer_outputs_innvestigate_wo_relu[ind])
        
        layer.statistics = stat
        ind += 1

In [22]:
# compute patterns
patternnet_pytorch.compute_patterns()
# set patterns
patternnet_pytorch.set_patterns()

In [23]:
def revert_tensor(tensor, axis=0):
    idx = [i for i in range(tensor.size(axis)-1,-1,-1)]
    idx = torch.LongTensor(idx)
    return tensor.index_select(axis,idx)

In [24]:
# get patterns
patterns_pytorch_linear, patterns_pytorch_aplus = [], []
patterns_conv_original_linear, patterns_conv_original_aplus = [], []
# save patterns in the two pattern lists and reshape them to have the same
# pattern values at the same place as in the innvestigate patterns
for layer in patternnet_pytorch.backward_layers[::-1]:
    layerclass = layer.__class__.__name__
    if layerclass in ['PatternConv2d', 'PatternLinear']:
        pattern_dict = layer.patterns
        pattern_linear = pattern_dict['A_linear']
        pattern_aplus = pattern_dict['A_plus']

        # dense pattern
        if pattern_linear.dim() == 2:
            # transpose to get same shape as innvestigate patterns
            pattern_linear = pattern_linear.permute(1, 0)
            pattern_aplus = pattern_aplus.permute(1, 0)
        # or convolutional pattern
        elif pattern_linear.dim() == 4:
            patterns_conv_original_linear.append(pattern_linear.numpy())
            patterns_conv_original_aplus.append(pattern_aplus.numpy())
            pattern_linear = pattern_linear.permute(2, 3, 0, 1)
            pattern_linear = revert_tensor(revert_tensor(pattern_linear, 1), 0)
            pattern_aplus = pattern_aplus.permute(2, 3, 0, 1)
            pattern_aplus = revert_tensor(revert_tensor(pattern_aplus, 1), 0)

        patterns_pytorch_linear.append(pattern_linear.numpy())
        patterns_pytorch_aplus.append(pattern_aplus.numpy())

In [25]:
def transpose_tensors_from_dict(dict):
    ''' transposes 0-th and 1-st dimension of all tensosr in the dictionary.
        After that converts them to numpy arrays.
    '''
    for key in dict:
        if key == "cnt":
            continue
        dict[key] = dict[key].permute(1,0).numpy()
    return dict

In [26]:
# get statistics
stats_pytorch_linear, stats_pytorch_aplus = [], []

# save statistics into two seperate lists for the linear and a-plus-minus pattern 
# and reshape them to be comparable to the innvestigate statistics
for layer in patternnet_pytorch.backward_layers:
    layerclass = layer.__class__.__name__
    if layerclass in ['PatternConv2d', 'PatternLinear']:
        
        stats_dict = layer.statistics
        stats_dict_linear = transpose_tensors_from_dict(stats_dict['linear'])
        stats_pytorch_linear.append(stats_dict_linear)
        stats_dict_aplus = transpose_tensors_from_dict(stats_dict['positive'])
        stats_pytorch_aplus.append(stats_dict_aplus)

# Create table for comparisons

In [27]:
row_names = [['Ex', 'Ey', 'Exy','Patterns'],['MAE', 'MSE', 'MaxDiff']]
rows = pd.MultiIndex.from_product(row_names)
col_names = [[PATTERN_TYPE],['Conv1', 'Conv2', 'Dense1', 'Dense2']]
columns = pd.MultiIndex.from_product(col_names)

df = pd.DataFrame(np.ones((12,4)), index=rows, columns=columns)

# Compare statistics

In [28]:
def mean_absolute_error(array1, array2):
    return np.mean(np.abs(array1 - array2))

def mean_squared_error(array1, array2):
    return np.mean((array1 - array2)**2)

def maximum_error(array1, array2):
    return np.max(np.abs(array1 - array2))

def all_close(array1, array2, tol=1e-5):
    return np.allclose(array1, array2, rtol=tol)

def mae_mse_ma(array1, array2):
    mae = mean_absolute_error(array1, array2)
    mse = mean_squared_error(array1, array2)
    md = maximum_error(array1, array2)
     
    print('Mean absolute error:', mae)
    print('Mean squared error:', mse)
    print('Maximum absolute difference:', md)
    
    return mae, mse, md

def relative_errors(array1, array2, rel_to_max=True):
    max_arr1 = np.max(np.abs(array1))
    max_arr2 = np.max(np.abs(array2))
    print('Maximum of arrays:', max_arr1, max_arr2)
    mae_rel = mean_absolute_error(array1, array2) / max(max_arr1, max_arr2) 
    md_rel = maximum_error(array1, array2) / max(max_arr1, max_arr2)
    
    print('Mean absolute error relative to maximum value:', mae_rel)
    print('Maximum difference relative to maximum value:', md_rel)

In [29]:
# note: order is always different
order_innvestigate = []
for i in range(4):
    for j in range(4):
        if stats_innvestigate[j]['e_x'].shape == stats_pytorch_aplus[i]['e_x'].shape:
            order_innvestigate.append(j)

keys = ['Conv1', 'Conv2', 'Dense1', 'Dense2']
if PATTERN_TYPE == 'relu':
    stats_pytorch = stats_pytorch_aplus
else:
    stats_pytorch = stats_pytorch_linear

for i in range(4):
    print('Statistics of layer %s:' %keys[i])
    print('Mean over x:')
    stats_inn_ex = stats_innvestigate[order_innvestigate[i]]['e_x']
    stats_py_ex = stats_pytorch[i]['e_x']
    mae, mse, md = mae_mse_ma(stats_inn_ex, stats_py_ex)
    df[PATTERN_TYPE,keys[i]].loc['Ex','MAE'] = mae
    df[PATTERN_TYPE,keys[i]].loc['Ex','MSE'] = mse
    df[PATTERN_TYPE,keys[i]].loc['Ex','MaxDiff'] = md
    print()
    
    print('Mean over y:')
    stats_inn_ey = stats_innvestigate[order_innvestigate[i]]['e_y']
    stats_py_ey = stats_pytorch[i]['e_y']
    if len(stats_py_ey.shape) == 2 and len(stats_inn_ey.shape) == 1:
        stats_py_ey = stats_py_ey[:,0]
    mae, mse, md = mae_mse_ma(stats_inn_ey, stats_py_ey)
    df[PATTERN_TYPE,keys[i]].loc['Ey','MAE'] = mae
    df[PATTERN_TYPE,keys[i]].loc['Ey','MSE'] = mse
    df[PATTERN_TYPE,keys[i]].loc['Ey','MaxDiff'] = md
    print()
    print('Mean over x*y:')
    stats_inn_exy = stats_innvestigate[order_innvestigate[i]]['e_xy']
    stats_py_exy = stats_pytorch[i]['e_xy']
    mae, mse, md = mae_mse_ma(stats_inn_exy, stats_py_exy)
    df[PATTERN_TYPE,keys[i]].loc['Exy','MAE'] = mae
    df[PATTERN_TYPE,keys[i]].loc['Exy','MSE'] = mse
    df[PATTERN_TYPE,keys[i]].loc['Exy','MaxDiff'] = md
    print()

Statistics of layer Conv1:
Mean over x:
Mean absolute error: 0.0
Mean squared error: 0.0
Maximum absolute difference: 0.0

Mean over y:
Mean absolute error: 0.0
Mean squared error: 0.0
Maximum absolute difference: 0.0

Mean over x*y:
Mean absolute error: 0.0
Mean squared error: 0.0
Maximum absolute difference: 0.0

Statistics of layer Conv2:
Mean over x:
Mean absolute error: 0.0
Mean squared error: 0.0
Maximum absolute difference: 0.0

Mean over y:
Mean absolute error: 0.0
Mean squared error: 0.0
Maximum absolute difference: 0.0

Mean over x*y:
Mean absolute error: 0.0
Mean squared error: 0.0
Maximum absolute difference: 0.0

Statistics of layer Dense1:
Mean over x:
Mean absolute error: 1.58346e-08
Mean squared error: 1.47631e-15
Maximum absolute difference: 7.45058e-07

Mean over y:
Mean absolute error: 0.0
Mean squared error: 0.0
Maximum absolute difference: 0.0

Mean over x*y:
Mean absolute error: 4.40836e-09
Mean squared error: 1.2102e-16
Maximum absolute difference: 1.93715e-07

S

In [30]:
# note: order is always different
order_innvestigate = []
for i in range(4):
    for j in range(4):
        if stats_innvestigate[j]['e_x'].shape == stats_pytorch_aplus[i]['e_x'].shape:
            order_innvestigate.append(j)

keys = ['Conv1', 'Conv2', 'Dense1', 'Dense2']
if PATTERN_TYPE == 'relu':
    stats_pytorch = stats_pytorch_aplus
else:
    stats_pytorch = stats_pytorch_linear

for i in range(4):
    print('Statistics of layer %s:' %keys[i])
    print('Mean over x:')
    stats_inn_ex = stats_innvestigate[order_innvestigate[i]]['e_x']
    stats_py_ex = stats_pytorch[i]['e_x']
    relative_errors(stats_inn_ex, stats_py_ex)
    print('Relative difference smaller 1e-5:', all_close(stats_inn_ex, stats_py_ex))

    print()
    
    print('Mean over y:')
    stats_inn_ey = stats_innvestigate[order_innvestigate[i]]['e_y']
    stats_py_ey = stats_pytorch[i]['e_y']
    if len(stats_py_ey.shape) == 2 and len(stats_inn_ey.shape) == 1:
        stats_py_ey = stats_py_ey[:,0]
    relative_errors(stats_inn_ey, stats_py_ey)
    print('Relative difference smaller 1e-5:', all_close(stats_inn_ey, stats_py_ey))

    print()
    print('Mean over x*y:')
    stats_inn_exy = stats_innvestigate[order_innvestigate[i]]['e_xy']
    stats_py_exy = stats_pytorch[i]['e_xy']
    relative_errors(stats_inn_exy, stats_py_exy)
    print('Relative difference smaller 1e-5:', all_close(stats_inn_exy, stats_py_exy))
    print()

Statistics of layer Conv1:
Mean over x:
Maximum of arrays: 10.6749 10.6749
Mean absolute error relative to maximum value: 0.0
Maximum difference relative to maximum value: 0.0
Relative difference smaller 1e-5: True

Mean over y:
Maximum of arrays: 9.23868 9.23868
Mean absolute error relative to maximum value: 0.0
Maximum difference relative to maximum value: 0.0
Relative difference smaller 1e-5: True

Mean over x*y:
Maximum of arrays: 293.097 293.097
Mean absolute error relative to maximum value: 0.0
Maximum difference relative to maximum value: 0.0
Relative difference smaller 1e-5: True

Statistics of layer Conv2:
Mean over x:
Maximum of arrays: 3.00052 3.00052
Mean absolute error relative to maximum value: 0.0
Maximum difference relative to maximum value: 0.0
Relative difference smaller 1e-5: True

Mean over y:
Maximum of arrays: 8.33097 8.33097
Mean absolute error relative to maximum value: 0.0
Maximum difference relative to maximum value: 0.0
Relative difference smaller 1e-5: True


# Compare patterns

In [31]:
layer_names = ['Conv1', 'Conv2', 'Dense1', 'Dense2']

if PATTERN_TYPE == 'relu':
    patterns_pytorch = patterns_pytorch_aplus
else:
    patterns_pytorch = patterns_pytorch_linear
    
for i in range(4):
    print('Layer: %s, Patterns of shape:' %layer_names[i], patterns_innvestigate[i].shape)
    pattern_inn = patterns_innvestigate[i]
    pattern_py = patterns_pytorch[i]
    mae, mse, md = mae_mse_ma(pattern_inn, pattern_py)
    df[PATTERN_TYPE,keys[i]].loc['Patterns','MAE'] = mae
    df[PATTERN_TYPE,keys[i]].loc['Patterns','MSE'] = mse
    df[PATTERN_TYPE,keys[i]].loc['Patterns','MaxDiff'] = md
    print()

Layer: Conv1, Patterns of shape: (3, 3, 1, 32)
Mean absolute error: 2.19463e-05
Mean squared error: 7.95975e-09
Maximum absolute difference: 0.000641346

Layer: Conv2, Patterns of shape: (3, 3, 32, 64)
Mean absolute error: 6.38233e-08
Mean squared error: 2.00037e-14
Maximum absolute difference: 2.86102e-06

Layer: Dense1, Patterns of shape: (9216, 512)
Mean absolute error: 2.4696e-09
Mean squared error: 5.07939e-15
Maximum absolute difference: 2.86102e-05

Layer: Dense2, Patterns of shape: (512, 10)
Mean absolute error: 1.02329e-07
Mean squared error: 2.89046e-13
Maximum absolute difference: 7.15256e-06



In [32]:
layer_names = ['Conv1', 'Conv2', 'Dense1', 'Dense2']

if PATTERN_TYPE == 'relu':
    patterns_pytorch = patterns_pytorch_aplus
else:
    patterns_pytorch = patterns_pytorch_linear
    
for i in range(4):
    print('Layer: %s, Patterns of shape:' %layer_names[i], patterns_innvestigate[i].shape)
    pattern_inn = patterns_innvestigate[i]
    pattern_py = patterns_pytorch[i]
    relative_errors(pattern_inn, pattern_py)
    print('Relative difference smaller 1e-5:', all_close(pattern_inn, pattern_py))

    print()

Layer: Conv1, Patterns of shape: (3, 3, 1, 32)
Maximum of arrays: 5.85551 5.85517
Mean absolute error relative to maximum value: 3.74798e-06
Maximum difference relative to maximum value: 0.000109529
Relative difference smaller 1e-5: False

Layer: Conv2, Patterns of shape: (3, 3, 32, 64)
Maximum of arrays: 5.38398 5.38398
Mean absolute error relative to maximum value: 1.18543e-08
Maximum difference relative to maximum value: 5.31395e-07
Relative difference smaller 1e-5: True

Layer: Dense1, Patterns of shape: (9216, 512)
Maximum of arrays: 20.612 20.612
Mean absolute error relative to maximum value: 1.19814e-10
Maximum difference relative to maximum value: 1.38804e-06
Relative difference smaller 1e-5: True

Layer: Dense2, Patterns of shape: (512, 10)
Maximum of arrays: 6.15189 6.1519
Mean absolute error relative to maximum value: 1.66337e-08
Maximum difference relative to maximum value: 1.16266e-06
Relative difference smaller 1e-5: True



# Print all differences

In [33]:
df

relu                                          
                         Conv1         Conv2        Dense1        Dense2
Ex       MAE      0.000000e+00  0.000000e+00  1.583462e-08  4.269063e-07
         MSE      0.000000e+00  0.000000e+00  1.476314e-15  5.440878e-13
         MaxDiff  0.000000e+00  0.000000e+00  7.450581e-07  2.563000e-06
Ey       MAE      0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
         MSE      0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
         MaxDiff  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
Exy      MAE      0.000000e+00  0.000000e+00  4.408359e-09  6.024541e-07
         MSE      0.000000e+00  0.000000e+00  1.210199e-16  3.143174e-12
         MaxDiff  0.000000e+00  0.000000e+00  1.937151e-07  8.463860e-06
Patterns MAE      2.194634e-05  6.382333e-08  2.469599e-09  1.023289e-07
         MSE      7.959753e-09  2.000368e-14  5.079395e-15  2.890459e-13
         MaxDiff  6.413460e-04  2.861023e-06  2.861023e-05  7.152557e-06

In [34]:
# df.to_csv('./dataframes/statsdiff.csv')